In [1]:
from Bisection import bisection
from NewtonRaphson import Newton_Raphson
from Secant import secant_method
from func import func_x, func_x_der

In [6]:
%%time
bisection(-2, 2, func_x)


CPU times: user 11 µs, sys: 3 µs, total: 14 µs
Wall time: 16.7 µs


-1.0

In [3]:
%%time
secant_method(func_x_der, 0, 0.2)

CPU times: user 12 µs, sys: 3 µs, total: 15 µs
Wall time: 17.2 µs


(0.486389035934543, 6)

In [4]:
%%time
Newton_Raphson(func_x_der, 0.2)

CPU times: user 14 µs, sys: 4 µs, total: 18 µs
Wall time: 19.6 µs


(0.486389035934477, 14)

In [ ]:
# oPTIM CUAR HAFM KO RANGF BUOOJC
# Toocs do hoi tu thuat toan cho cac phuong trinhf nhieu bien so
